<a href="https://colab.research.google.com/github/RomaTruba/dzSEM/blob/main/s_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers tensorflow

Импорт библиотек

In [ ]:
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
import torch

Загрузка предобученной модели и токенизатора и создание конвейера для вопросно-ответной системы

In [ ]:

model_name = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer
)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:

context = """
Hugging Face — компания, разрабатывающая инструменты для обработки естественного языка (NLP).
Они известны своей библиотекой Transformers, которая предоставляет тысячи предобученных моделей
для задач классификации текста, ответов на вопросы и генерации текста.
Компания основана в 2016 году и базируется в Нью-Йорке.
"""

questions = [
    "Когда была основана Hugging Face?",
    "Где находится Hugging Face?",
    "Для чего используется библиотека Transformers?"
]

Получение ответов на вопросы

In [ ]:

for question in questions:
    result = qa_pipeline(question=question, context=context)
    print(f"Вопрос: {question}")
    print(f"Ответ: {result['answer']} (уверенность: {result['score']:.3f})")
    print("-" * 50)

Вопрос: Когда была основана Hugging Face?
Ответ: 2016 году (уверенность: 0.051)
--------------------------------------------------
Вопрос: Где находится Hugging Face?
Ответ: генерации текста (уверенность: 0.000)
--------------------------------------------------
Вопрос: Для чего используется библиотека Transformers?
Ответ: своей (уверенность: 0.001)
--------------------------------------------------


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:

documents = [
    "Hugging Face разрабатывает инструменты для NLP, включая библиотеку Transformers.",
    "Transformers предоставляет предобученные модели для обработки текста.",
    "Компания основана в 2016 году в Нью-Йорке."
]

questions = [
    "Когда была основана Hugging Face?",
    "Какие инструменты разрабатывает Hugging Face?"
]

Создание TF-IDF векторизатора

In [ ]:

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents)

Функция для поиска ответа

In [ ]:

def tfidf_answer(question):
    question_vec = vectorizer.transform([question])
    similarities = cosine_similarity(question_vec, doc_vectors)
    best_doc_idx = np.argmax(similarities)
    return documents[best_doc_idx]

Тестирование

In [ ]:

for question in questions:
    answer = tfidf_answer(question)
    print(f"Вопрос: {question}")
    print(f"Ответ (TF-IDF): {answer}")
    print("-" * 50)

Вопрос: Когда была основана Hugging Face?
Ответ (TF-IDF): Hugging Face разрабатывает инструменты для NLP, включая библиотеку Transformers.
--------------------------------------------------
Вопрос: Какие инструменты разрабатывает Hugging Face?
Ответ (TF-IDF): Hugging Face разрабатывает инструменты для NLP, включая библиотеку Transformers.
--------------------------------------------------


In [ ]:
from sentence_transformers import SentenceTransformer


Загрузка модели для эмбеддингов

In [ ]:

embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Получение эмбеддингов для документов

In [ ]:

doc_embeddings = embedding_model.encode(documents)

Функция для ответа на основе эмбеддингов

In [ ]:

def embedding_answer(question):
    question_embedding = embedding_model.encode([question])
    similarities = cosine_similarity(question_embedding, doc_embeddings)
    best_doc_idx = np.argmax(similarities)
    return documents[best_doc_idx]

Сравнение

In [ ]:

for question in questions:
    tfidf_ans = tfidf_answer(question)
    emb_ans = embedding_answer(question)
    print(f"Вопрос: {question}")
    print(f"TF-IDF: {tfidf_ans}")
    print(f"Эмбеддинги: {emb_ans}")
    print("-" * 50)

Вопрос: Когда была основана Hugging Face?
TF-IDF: Hugging Face разрабатывает инструменты для NLP, включая библиотеку Transformers.
Эмбеддинги: Hugging Face разрабатывает инструменты для NLP, включая библиотеку Transformers.
--------------------------------------------------
Вопрос: Какие инструменты разрабатывает Hugging Face?
TF-IDF: Hugging Face разрабатывает инструменты для NLP, включая библиотеку Transformers.
Эмбеддинги: Hugging Face разрабатывает инструменты для NLP, включая библиотеку Transformers.
--------------------------------------------------


# ВЫВОД:TF-IDF подходит для простых задач с четкими ключевыми словами.

# Эмбеддинги лучше для сложных вопросов, где важна семантика.

# Hugging Face Transformers (как в Задании №2) — наиболее продвинутый вариант, но требует больше ресурсов.